In [1]:
import pandas as pd
import torch
from torch import nn
from torch.utils.data import Dataset, DataLoader

In [2]:
gameStateDF = pd.read_csv('gameStateDF.csv')

In [3]:
class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.features = dataframe[["timer", "health_1", "x_coord_1", "y_coord_1", "health_2", "x_coord_2", "y_coord_2", "Y_2", "B_2"]].values
        self.outputs = dataframe[["Y_1", "B_1"]].values

    def __len__(self):
        return len(self.features)

    def __getitem__(self, idx):
        return self.features[idx], self.outputs[idx]

In [7]:
dataset = CustomDataset(gameStateDF)

In [4]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(9, 64)  # Input size: 9 features, output size: 64
        self.fc2 = nn.Linear(64, 2)  # Input size: 64, output size: 2

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [8]:
model = NeuralNet()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_size, val_size])

batch_size = 32
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

In [ ]:
num_epochs = 10
for epoch in range(num_epochs):
    model.train()  # Set the model to training mode
    for features, outputs in train_loader:
        optimizer.zero_grad()  # Clear gradients
        predictions = model(features.float())  # Forward pass
        loss = criterion(predictions, outputs.float())  # Calculate the loss
        loss.backward()  # Backward pass
        optimizer.step()  # Update weights

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        val_loss = 0.0
        for features, outputs in val_loader:
            predictions = model(features.float())  # Forward pass
            val_loss += criterion(predictions, outputs.float()).item()  # Calculate the loss

    # Calculate average validation loss
    val_loss /= len(val_loader)

    # Print the loss for each epoch
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item():.4f}, Val Loss: {val_loss:.4f}")


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found object

Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Program Files\Python310\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly closed by the remote host
Exception in callback BaseSelectorEventLoop._read_from_self()
handle: <Handle BaseSelectorEventLoop._read_from_self()>
Traceback (most recent call last):
  File "c:\Program Files\Python310\lib\asyncio\events.py", line 80, in _run
    self._context.run(self._callback, *self._args)
  File "c:\Program Files\Python310\lib\asyncio\selector_events.py", line 120, in _read_from_self
    data = self._ssock.recv(4096)
ConnectionResetError: [WinError 10054] An existing connection was forcibly 

In [ ]:
test_dataset = CustomDataset(gameStateDF)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
model.eval()
predictions = []
with torch.no_grad():
    for features, _ in test_loader:
        batch_predictions = model(features.float())
        predictions.extend(batch_predictions.numpy())